In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Data

In [2]:
from data_utils.data_loaders import FocusCNNLoader

/home/ubuntu/focus-convolutional-neural-network/venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
loader = FocusCNNLoader(
    batch_size=16,
    shuffle=True,
    num_workers=1,
    images_dir = "/home/ubuntu/focus-convolutional-neural-network/data/processed/COCO-2017/CocoFocusCNN/train/images",
    csv_path="/home/ubuntu/focus-convolutional-neural-network/data/processed/COCO-2017/CocoFocusCNN/train/labels.csv",
    labels={
        "0": "none",
        "1": "person",
        "2": "car",
        "3": "bicycle"
    },
    tf_image_size=(640, 640),
    save_out_dir="/home/ubuntu/focus-convolutional-neural-network/res/focus_cnn/coco_focuscnn/trainer/labels/",
    validation_split=0.15,
    is_test=False
)

In [4]:
dl = loader.get_train_loader()

In [5]:
iter_dl = iter(dl)
data = next(iter_dl)

In [25]:
data[2]

{'1': {'label': tensor([0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 1., 0., 0., 1.]),
  'transform': tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 1.0314e-01, -5.3048e-01, -1.5887e-01],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 9.2952e-01, -4.8666e-01, -5.2706e+00],
          [ 1.6430e-01, -4.9750e-02, -6.4370e-01],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 6.8483e-01,  3.4219e-03, -2.1521e+00],
          [ 2.2016e-02,  1.1886e-01, -1.4836e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 1.9833e-01,  6.5637e-01, -2.7717e-01],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 6.0781e-02,  8.1197e-01, -3.0236e+00]])},
 '2': {'label': tensor([0., 0., 0., 0., 2., 2., 2., 0., 0., 0., 2., 0., 0., 0.

# Models

In [4]:
import torch
from pathlib import Path
from models import ResNet34Classifier, ResFocusNetwork, TransformModule
from utils import ConfigParser

In [5]:
config_cls_path = ""

In [6]:
cls_model = ResNet34Classifier(n_classes=4)
model_checkpoint = torch.load("/home/ubuntu/focus-convolutional-neural-network/res/classifiers/coco_classifier_multi/trainer/0116_044824/models/checkpoint-epoch44.pth")

cls_model.load_state_dict(model_checkpoint['state_dict'])

<All keys matched successfully>

In [7]:
focus_person_model = ResFocusNetwork(threshold=0.5, backbone="resnet34", loss_lambda_tr=1,loss_lambda_sc=1, loss_lambda_rot=3, inp_img_size=(640, 640), loss_rot=False)
model_checkpoint = torch.load("/home/ubuntu/focus-convolutional-neural-network/res/focus/coco_focus_person/trainer/0103_092318/models/checkpoint-epoch79.pth")

focus_person_model.load_state_dict(model_checkpoint['state_dict'])

<All keys matched successfully>

In [8]:
focus_car_model = ResFocusNetwork(threshold=0.5, backbone="resnet34", loss_lambda_tr=1,loss_lambda_sc=1, loss_lambda_rot=3, inp_img_size=(640, 640), loss_rot=False)
model_checkpoint = torch.load("/home/ubuntu/focus-convolutional-neural-network/res/focus/coco_focus_car/trainer/0111_050401/models/checkpoint-epoch19.pth")

focus_car_model.load_state_dict(model_checkpoint['state_dict'])

<All keys matched successfully>

In [9]:
focus_bicycle_model = ResFocusNetwork(threshold=0.5, backbone="resnet34", loss_lambda_tr=1,loss_lambda_sc=1, loss_lambda_rot=3, inp_img_size=(640, 640), loss_rot=False)
model_checkpoint = torch.load("/home/ubuntu/focus-convolutional-neural-network/res/focus/coco_focus_bicycle/trainer/0114_175807/models/checkpoint-epoch56.pth")

focus_bicycle_model.load_state_dict(model_checkpoint['state_dict'])

<All keys matched successfully>

In [10]:
from models.transform.TransformModule import TransformModule

In [11]:
tf_module = TransformModule(image_out_sz=(300, 300))

In [12]:
dl_train = loader.get_train_loader()

In [34]:
# cls_model.eval()
# focus_person_model.eval()
# focus_car_model.eval()
# focus_bicycle_model.eval()

# with torch.no_grad():
#     data = next(iter(dl_train))
#     focus_person_model.to(device="cuda")
#     focus_car_model.to(device="cuda")
#     cls_model.to(device="cuda")
#     inp_img = data[0].to(device="cuda")
#     for k,v in data[2]['1'].items():
#         data[2]['1'][k] = v.to(device="cuda")
#         data[2]['2'][k] = v.to(device="cuda")

#     out_focus_person = focus_person_model(inp_img, data[2]['1'])
#     out_focus_car = focus_car_model(inp_img, data[2]['2'])

#     out_focus_person_cat = torch.cat([out_focus_person['out_translate_x'], out_focus_person['out_translate_y'], out_focus_person['out_scale'], out_focus_person['out_rotate']],dim = 1)
    
#     out_focus_car_cat = torch.cat([out_focus_car['out_translate_x'], out_focus_car['out_translate_y'], out_focus_car['out_scale'], out_focus_car['out_rotate']],dim = 1)
    
#     inp_cls_person = tf_module(inp_img, out_focus_person_cat)
#     inp_cls_car = tf_module(inp_img, out_focus_car_cat)
    
#     target_cls = torch.cat([data[2]['1']['label'],data[2]['2']['label']])
    
#     inp_cls = torch.cat([inp_cls_person, inp_cls_car])
    
#     out_cls = cls_model(inp_cls)
    
    
    

In [13]:
loss = torch.nn.CrossEntropyLoss()

In [14]:
iter_dl = iter(dl_train)
bn_1 = next(iter_dl)
bn_2 = next(iter_dl)

In [15]:
cls_model.train()
focus_person_model.train()
focus_car_model.train()
focus_bicycle_model.train()

data = bn_2
focus_person_model.to(device="cuda")
focus_car_model.to(device="cuda")
focus_bicycle_model.to(device="cuda")
cls_model.to(device="cuda")
inp_img = data[0].to(device="cuda")
for k in data[2]['1'].keys():
    data[2]['1'][k] = data[2]['1'][k].to(device="cuda")
    data[2]['2'][k] = data[2]['2'][k].to(device="cuda")
    data[2]['3'][k] = data[2]['3'][k].to(device="cuda")

out_focus_person = focus_person_model(inp_img, data[2]['1'])
out_focus_car = focus_car_model(inp_img, data[2]['2'])
out_focus_bicycle = focus_bicycle_model(inp_img, data[2]['3'])

out_focus_person_cat = torch.cat([out_focus_person['out_translate_x'], out_focus_person['out_translate_y'], out_focus_person['out_scale'], out_focus_person['out_rotate']],dim = 1)
out_focus_car_cat = torch.cat([out_focus_car['out_translate_x'], out_focus_car['out_translate_y'], out_focus_car['out_scale'], out_focus_car['out_rotate']],dim = 1)
out_focus_bicycle_cat = torch.cat([out_focus_bicycle['out_translate_x'], out_focus_bicycle['out_translate_y'], out_focus_bicycle['out_scale'], out_focus_bicycle['out_rotate']],dim = 1)
    
inp_cls_person = tf_module(inp_img, out_focus_person_cat)
inp_cls_car = tf_module(inp_img, out_focus_car_cat)
inp_cls_bicycle = tf_module(inp_img, out_focus_bicycle_cat)
    
target_cls = torch.cat([data[2]['1']['label'],data[2]['2']['label'],data[2]['3']['label']]).type(torch.LongTensor).to(device="cuda")
    
inp_cls = torch.cat([inp_cls_person, inp_cls_car,inp_cls_bicycle])
    
out_cls = cls_model(inp_cls)

loss_val = loss(out_cls, target_cls)

loss_val.backward()

    
    

/home/ubuntu/focus-convolutional-neural-network/src/pipeline/loss.py:112: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3277.)
  * target_cls.T


In [16]:
loss_val

tensor(0.9681, device='cuda:0', grad_fn=<NllLossBackward0>)

In [20]:
loss_val.backward()

In [17]:
out_cls

tensor([[9.7316e-01, 7.8600e-01, 1.4300e-02, 1.4229e-02],
        [4.7360e-10, 1.0000e+00, 3.2201e-10, 8.4702e-12],
        [9.9999e-01, 6.0839e-03, 1.0791e-04, 3.1950e-04],
        [6.6723e-08, 1.0000e+00, 2.8146e-10, 1.4501e-10],
        [1.0000e+00, 2.8351e-03, 3.6885e-04, 1.7107e-04],
        [8.2177e-01, 9.6585e-01, 2.1397e-02, 6.0461e-03],
        [9.9991e-01, 3.3740e-03, 2.1063e-03, 3.1507e-03],
        [8.7184e-03, 9.9955e-01, 4.0898e-03, 4.8187e-03],
        [1.0000e+00, 8.6415e-04, 7.1371e-04, 1.1578e-03],
        [2.7567e-12, 1.0000e+00, 1.7300e-16, 1.7821e-14],
        [7.1782e-07, 9.9999e-01, 1.3230e-05, 1.1679e-03],
        [9.5203e-01, 6.7700e-01, 2.1367e-02, 5.1367e-03],
        [1.0000e+00, 1.1474e-04, 1.1643e-03, 1.6977e-03],
        [7.7516e-01, 7.2998e-01, 7.2511e-02, 2.7965e-02],
        [5.8405e-01, 9.9958e-01, 1.6087e-04, 3.5766e-04],
        [3.0637e-10, 1.0000e+00, 3.2563e-08, 7.9242e-12],
        [9.9469e-01, 1.8517e-02, 1.5511e-01, 3.8700e-03],
        [1.885

In [18]:
target_cls

tensor([0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 2, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       device='cuda:0')

In [19]:
torch.argmax(out_cls, dim=1)

tensor([0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 3, 0],
       device='cuda:0')